In [ ]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

class Layer_Dense:

    #layer initialization 
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))

    #forward pass
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        #derivative wrt of weights is inputs. Dimension adjustment is needed
        self.dweights = np.dot(self.inputs.T, dvalues)
        #derivative of bias is column sums
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        #derivative wrt of inputs is weights
        self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0,inputs) #if the inputs is lower than 0, we make it 0, uf not, then we pass on

    def backward(self, dvalues):
        self.dinputs = dvalues.copy() #copy the gradient matrix
        self.dinputs[self.inputs<= 0] = 0 #if it is lower than 0 then we'll make it zero

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True)) #make the max = 1 and the min is -inf
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)# normalize

        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues)  in enumerate(zip(self.output, dvalues)):
            #single output is self.output[0]. it has the dimension 1xn, then the next line, we'll shift if to n,1
            single_output = single_output.reshape(-1,1) #reshape the output to become (,1). n row with 1 column
            
            #diagflat is to craete the matrix where diagonal is a value and the rest is 0. then based on the formula 
            #diagflat will have n,n matrix with diagonal is single_output and the rest is 0
            #then we have single.output where dim = n,1 and single.output.T dim = 1,n. this will result n,n
            jacobian_matrix  = np.diagflat(single_output) - np.dot(single_output, single_output.T)

            #jacobian matrix is n,n and single_values is n,1. Why Dvalues has n,n dimension and single is only the first row
            #this will result n,1 dimension
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output,y) #output in here is the preddiction
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        #clip data to avoid division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1 : 
            #softmax_outputs1 = np.array([[ 0.7 , 0.1 , 0.2 ],
            #[ 0.1 , 0.5 , 0.4 ],
            #[ 0.02 , 0.9 , 0.08 ]])
            #class_targets1 = np.array([ 0 , 1 , 1 ])
            #for example [0,1,1] just go to observation where it is y true. 
            #y_pred has a dimension nxn so for the first row, take index 0 y_pred_clipped[0,0]
            #y_pred_clipped[1,1]
            #y_pred_clipped[2,1]

            correct_cofidences = y_pred_clipped[ range(samples), y_true]   

        elif len(y_true.shape) == 2:
            #softmax_outputs2 = np.array([[ 0.7 , 0.1 , 0.2 ],
            #[ 0.1 , 0.5 , 0.4 ],
            #[ 0.02 , 0.9 , 0.08 ]])
            #class_targets2 = np.array([[ 1 , 0 , 0 ],
            #[ 0 , 1 , 0 ],
            #[ 0 , 1 , 0 ]])
            #since this is one hot encoding. only 1 value is 1 and the rest is zero. so when multiplying, only the given 1 will yield a result
            correct_confidences1 = np.sum(y_pred_clipped * y_true, axis = 1) 
        
        negative_log_likelihoods = -np.log(correct_cofidences)
        return negative_log_likelihoods
    
    def backward(self, dvalues, y_true):
        #number of samples
        samples = len(dvalues) #np.array([[ 1 , 2 , 3 ],[ 5 , 6 , 7 ],[ 9 , 10 , 11  ]])

        labels = len(dvalues[0]) #array([1,2,3])

        #if y_true is [0,1,1]
        #then np.eye will make it 
        #array([[1., 0., 0.],
        #       [0., 1., 0.],
        #       [0., 1., 0.]], dtype=float32)

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true] 
    
        self.dinputs = -y_true / dvalues #partial derivatives with respect tp inputs = matrix 3x3 - 3x3
        #the derivative of this loss fucntion with respect ot is input = ground truth vector / vector of predicted values

        self.dinputs = self.dinputs / samples
        #normalize to make the sum magnitude invariant to the number of samples. 

        
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs) #use softmax activation
        self.output = self.activation.output #the output is a probability
        return self.loss.calculate(self.output, y_true) #calculate loss between predicted (self.output) and y_true
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)

        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis = 1) #convert from one hot encoder to the discrete true labels
        
        self.dinputs = dvalues.copy()
        self.dinputs [range(samples) ,y_true] -= 1 #only at the given ytue, the value is minus by one. why?
        #becayse the partial derivative of loss wrt of softmax function inputs. 

class Optimizer_SGD:
    def __init__(self, learning_rate = 1):
        self.learning_rate = learning_rate

    def update_params(self, layer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases  


In [ ]:
X,y = spiral_data(samples =100, classes = 3)
f'Xshape = {X.shape} and yshape = {y.shape}'

dense1 = Layer_Dense(2,64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64,3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_SGD

for epoch in range(11):
    print(f'this is {epoch +1} iteration')
    dense1.forward(X)
    print(f'whats happening in dense1.forward(X) is we have X which has a dimension of {X.shape}')
    print(f'we initiate weight {dense1.weights.shape} and bias {dense1.biases.shape}')
    print(f'dot product of the input {X.shape} and weight {dense1.weights.shape}, makes it {dense1.output.shape} plus bias {dense1.biases.shape}')
    print(f'we will have hte result of {dense1.output.shape}\n')

    activation1.forward(dense1.output)
    print(f'this is the ReLU activation function. any value less than zero. we make it zero\n')

    dense2.forward(activation1.output)
    print(f'whats happening in dense2.forward(activation1.output) is we have activation1.output which has a dimension of {activation1.output.shape}')
    print(f'we initiate weight {dense2.weights.shape} and bias {dense2.biases.shape}')
    print(f'dot product of the input {X.shape} and weight {dense2.weights.shape}, makes it {dense2.output.shape} plus bias {dense2.biases.shape}')
    print(f'we will have the result of {dense2.output.shape}\n')

    print(f'loss = loss_activation.forward(dense2.output, y)')
    loss = loss_activation.forward(dense2.output, y)
    print(f'in this loss function, we use the softmax method which is ')
    print(f'we transform the inputs into -inf and 0 to avoid overflow')
    print(f' then we normalize dense2.output so we got the probabilities. which is yhat')
    print(f'dimension of yhat is {loss_activation.output.shape}')
    print(f' Then we calculate the difference between loss_activation.output and y (aka y_true) ')
    print(f' the calculation follow LossCategoricalCrossentropy. ')
    print(f' we have the length of y_true is {len(y.shape)}') 
    print(f'which is dense layer not one hot encoding')
    print(f' we got the correct confidences by taking each matrix from 0 to n, to its designtaed y_true. refer to the code\n')

    print(f'predictions = np.argmax(loss_activation.output, axis = 1 )')
    print(f'predictions is the highest number of predictions in each row')
    print(f' then we compare it with the true y using this formula if len (y.shape) == 2 :        y = np.argmax(y, axis = 1 )')
    print(f' which is if it is one hot encoding, we want to make them layer dense.')
    print(f' then we average the accuracy,we give points when predictions is equal to true_y')


    predictions = np.argmax(loss_activation.output, axis = 1 )
    if len (y.shape) == 2 :
        y = np.argmax(y, axis = 1 )
    accuracy = np.mean(predictions == y)
    print(f' acc:{accuracy}')


    loss_activation.backward(loss_activation.output, y)
    print('loss_activation.backward(loss_activation.output, y)')
    print('remember that we are looking for partial derivative of loss categorical cross entropy with respect to the inputs.')
    print('dvalues is loss_activaiton.output and partial derivative is yhat - y so that ')


    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    print(f'end of iteration \n \n')


    
